<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%88%98%EC%97%85/pytorch_CustomDataset_%ED%81%B4%EB%9E%98%EC%8A%A4_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch
!pip3 install torchvision
!pip install pillow
!pip install transformers
!pip install datasets

In [60]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn, optim
from torchvision import datasets
from PIL import Image
from torchvision.transforms import ToPILImage
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import zipfile
import os

# ZIP 파일 열기
with zipfile.ZipFile("/content/training_set (2).zip", "r") as zip_ref:
    # 파일 목록 가져오기
    file_list = zip_ref.namelist()

    # 파일 목록 출력
    for file in file_list:
        print(file)

        # 파일 압축해제
        zip_ref.extract(file)

In [80]:
# os.remove('/content/test_set/cats/.ipynb_checkpoints')
# os.remove('/content/test_set/dogs/.ipynb_checkpoints')
# os.rmdir('/content/test_set/cats/.ipynb_checkpoints')
# os.rmdir('/content/test_set/dogs/.ipynb_checkpoints')

In [35]:
# CustomDataset 클래스에서 이미지를 읽을 때 이미지를 PIL 이미지로 변환 후 텐서로 변환
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None): # root_dir = 폴더 경로, transform = 이미지 변환
        self.root_dir = root_dir # 폴더 경로 변수화
        self.transform = transform # 함수 변수화
        self.classes = os.listdir(root_dir) # 폴더 내 파일 리스트 변수화
        self.data = [] # 폴더 안의 파일 전체 경로, 이미지경로, 라벨 저장할 리스트
        for label in range(len(self.classes)): # 리스트 변수에서 개수로 라벨설정
            class_folder = os.path.join(root_dir, self.classes[label]) # 리스트변수[0], [1]의 폴더 명에 따라 이미지 폴더 전체 경로 생성
            for filename in os.listdir(class_folder): # 이미지 폴더 내 파일 리스트 가져와서 for문 돌면서 1개씩 가져옴
                img_path = os.path.join(class_folder, filename) # 이미지 파일 경로 설정
                self.data.append((img_path, label)) # data 리스트에 이미지 경로, 라벨 저장
    def __len__(self):
        return len(self.data) # data 리스트에서 개수 세기
    def __getitem__(self, idx): # 인덱스값 넣어주면 이미지와 라벨 가져오는 함수
        img_path, label = self.data[idx] # 이미지 경로와 라벨을 data 리스트에서 가져옴
        image = Image.open(img_path) # 이미지 경로에서 이미지 불러오기
        if self.transform:
            image = self.transform(image) # 이미지를 transform에 설정된 기준으로 가져오기
        return image, label # 이미지 라벨 출력

In [76]:
data_dir = "/content"  # 현재 작업 디렉토리로 설정
batch_size = 32
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CustomDataset(os.path.join(data_dir, 'training_set'), transform=transform)
test_dataset = CustomDataset(os.path.join(data_dir, 'test_set'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### 모델 CNN

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torch.utils.data import DataLoader

# CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 56 * 56, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
# 모델 초기화 및 손실 함수, 최적화 함수 설정
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')



Epoch 1, Loss: 0.7060408336775643
Epoch 2, Loss: 0.651059091091156
Epoch 3, Loss: 0.6024794152804783
Epoch 4, Loss: 0.5607253057616097


In [ ]:
# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')